In [1]:
# libs
from pulp import *

## Defining Problem

A company sells three types of product (prod_1, prod_2, prod_3), currently, she has no stock and does not want to have leftovers at the end of the month. The forecast for next month is the sale of 1200 units of prod_1, 980 of prod_2 and 2000 of prod_3.

The products prod_1, prod_2, prod_3 are sold for 500 reais, 350 reais, 150 reais, respectively. The cost of materials for each is 395 reais, 266 reais and 73 reais, respectively.

The company is relatively large, having, in one month, the capacity to carry out 17,000 hours of mounting and 6,200 hours of tests (fundamental steps in the construction of the product), where the hours are worth 14 reais and 19 reais, respectively.

The time taken to build the products is: prod_1 is 8 hours of mounting and 3 hours of testing; prod_2 is 6 hours of mounting and 2.5 hours of testing; prod_3 is 3 hours of mounting and 1 hour of testing.

### 1 Problem
The company wants to know how many units of each product it must produce (mounting and test) to maximize its net profit (none of the restrictions can be broken).

### 2 Problem
The company wants to know how many units of each product it must produce (mounting and test) to have a net profit of 500,000 reais and whether it already has the infrastructure to do so.

### Parameters

In [2]:
#Pqty
prod_1, prod_2, prod_3 = 1200, 980, 2000

#Sp
sale_1, sale_2, sale_3 = 600, 550, 150

#Cp
cost_1, cost_2, cost_3 = 395, 266, 73

#CMp
cmounting_1, cmounting_2, cmounting_3 = 14, 14, 14

#CTp
ctest_1, ctest_2, ctest_3 = 19, 19, 19

#HMp
hmounting_1, hmounting_2, hmounting_3 = 8, 6, 3

#HTp
htest_1, htest_2, htest_3 = 3, 2.5, 1

#Company capacity
mounting_max = 17000
test_max = 6200
profit_p2 = 50000

### Quantity Variables

In [3]:
x_prod1 = LpVariable("Unidades Prod1", 0, None, LpInteger)
x_prod2 = LpVariable("Unidades Prod2", 0, None, LpInteger)
x_prod3 = LpVariable("Unidades Prod3", 0, None, LpInteger)

### Cost Function

* profit = $\sum Pqty*Sp$ - $\sum Pqty*HMp*CMp$ - $\sum Pqty*Htp*CTp$ - $\sum Pqty*Cp$

In [4]:
revenues = (x_prod1*sale_1) + (x_prod2*sale_2) + (x_prod3*sale_3)
mounting_spend = (x_prod1*hmounting_1*cmounting_1) + (x_prod2*hmounting_2*cmounting_2) + (x_prod3*hmounting_3*cmounting_3)
test_spend = (x_prod1*htest_1*ctest_1) + (x_prod2*htest_2*ctest_2) + (x_prod3*htest_3*ctest_3)
cost_material = (x_prod1*cost_1) + (x_prod2*cost_2) + (x_prod3*cost_3)

In [5]:
cost_function = revenues - mounting_spend - test_spend - cost_material

## 1 Problem - Product Quantity

### Create the object problem

In [6]:
problem = LpProblem("MaxProfit", LpMaximize)
problem += cost_function
problem

MaxProfit:
MAXIMIZE
36*Unidades_Prod1 + 152.5*Unidades_Prod2 + 16*Unidades_Prod3 + 0.0
VARIABLES
0 <= Unidades_Prod1 Integer
0 <= Unidades_Prod2 Integer
0 <= Unidades_Prod3 Integer

### Create Constraints

In [7]:
# Max amount of mount/test time
problem += (x_prod1*hmounting_1) + (x_prod2*hmounting_2) +(x_prod3*hmounting_3) <= mounting_max,"Max Amount of Mounting Time"
problem += (x_prod1*htest_1) + (x_prod2*htest_2) +(x_prod3*htest_3) <= test_max,"Max Amount of Test Time"

In [8]:
# Production is equal or lower than the prediction
problem += x_prod1 <= prod_1,"Production 1 equal or lower"
problem += x_prod2 <= prod_2,"Production 2 equal or lower"
problem += x_prod3 <= prod_3,"Production 3 equal or lower"

In [9]:
# Final Problem
problem

MaxProfit:
MAXIMIZE
36*Unidades_Prod1 + 152.5*Unidades_Prod2 + 16*Unidades_Prod3 + 0.0
SUBJECT TO
Max_Amount_of_Mounting_Time: 8 Unidades_Prod1 + 6 Unidades_Prod2
 + 3 Unidades_Prod3 <= 17000

Max_Amount_of_Test_Time: 3 Unidades_Prod1 + 2.5 Unidades_Prod2
 + Unidades_Prod3 <= 6200

Production_1_equal_or_lower: Unidades_Prod1 <= 1200

Production_2_equal_or_lower: Unidades_Prod2 <= 980

Production_3_equal_or_lower: Unidades_Prod3 <= 2000

VARIABLES
0 <= Unidades_Prod1 Integer
0 <= Unidades_Prod2 Integer
0 <= Unidades_Prod3 Integer

### Resolve Problem

In [10]:
if problem.solve():
    print("Solution found!!!")
    print(f"Max profit possible: R${value(problem.objective)}")
    print(f"Quantity of prod_1: {problem.variables()[0].varValue} units")
    print(f"Quantity of prod_2: {problem.variables()[1].varValue} units")
    print(f"Quantity of prod_3: {problem.variables()[2].varValue} units")
else:
    print("No solution found!!")

Solution found!!!
Max profit possible: R$202442.0
Quantity of prod_1: 584.0 units
Quantity of prod_2: 980.0 units
Quantity of prod_3: 1998.0 units


## 2 Problem - Mount Time

### Create the object problem

In [11]:
problem = LpProblem("Profit500k", LpMinimize)
problem += cost_function
problem

Profit500k:
MINIMIZE
36*Unidades_Prod1 + 152.5*Unidades_Prod2 + 16*Unidades_Prod3 + 0.0
VARIABLES
0 <= Unidades_Prod1 Integer
0 <= Unidades_Prod2 Integer
0 <= Unidades_Prod3 Integer

### Create Constraints

In [12]:
# Profit need to be R$500.000
problem += cost_function == 500000, "Final"

In [13]:
# Final Problem
problem

Profit500k:
MINIMIZE
36*Unidades_Prod1 + 152.5*Unidades_Prod2 + 16*Unidades_Prod3 + 0.0
SUBJECT TO
Final: 36 Unidades_Prod1 + 152.5 Unidades_Prod2 + 16 Unidades_Prod3 = 500000

VARIABLES
0 <= Unidades_Prod1 Integer
0 <= Unidades_Prod2 Integer
0 <= Unidades_Prod3 Integer

### Resolve Problem

In [14]:
if problem.solve():
    print("Solution found!!!")
    print(f"Max profit possible: R${value(problem.objective)}")
    print(f"Quantity of prod_1: {problem.variables()[0].varValue} units")
    print(f"Quantity of prod_2: {problem.variables()[1].varValue} units")
    print(f"Quantity of prod_3: {problem.variables()[2].varValue} units")
    print(f"Amount of mounting time: {(problem.variables()[0].varValue*hmounting_1)+(problem.variables()[1].varValue*hmounting_2)+(problem.variables()[2].varValue*hmounting_3)} hours")
    print(f"Amount of test time: {(problem.variables()[0].varValue*htest_1)+(problem.variables()[1].varValue*htest_2)+(problem.variables()[2].varValue*htest_3)} hours")
else:
    print("No solution found!!")

Solution found!!!
Max profit possible: R$500000.0
Quantity of prod_1: 7.0 units
Quantity of prod_2: 3272.0 units
Quantity of prod_3: 48.0 units
Amount of mounting time: 19832.0 hours
Amount of test time: 8249.0 hours
